# RAG 실습: Web 데이터 로딩 & PromptTemplate 활용

이 노트북에서는 포켓몬스터 나무위키 페이지에서 데이터를 로드하여 RAG(검색 증강 생성) 실습을 진행합니다.  
PromptTemplate을 활용한 다양한 프롬프트 구성 방법도 함께 다룹니다.

---
**실습 목표**
- 웹에서 문서를 로드하고, 텍스트를 청크로 분할
- 임베딩 및 벡터 DB 구축
- 검색 기반 답변 생성(RAG) 체인 구성
- PromptTemplate과 LangChain Hub 프롬프트 활용법 이해
- RunnablePassthrough 등 체인 유틸리티 활용법 익히기

## 1. 환경 변수 불러오기

- OpenAI API 키 등 민감 정보는 .env 파일에 저장합니다.
- dotenv를 통해 환경 변수를 로드합니다.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

## 2. 웹 문서 로딩 및 텍스트 분할

- WebBaseLoader로 나무위키 포켓몬 타입 문서를 불러옵니다.
- RecursiveCharacterTextSplitter로 문서를 일정 길이의 청크로 분할합니다.
- 분할된 청크는 이후 임베딩 및 검색에 활용됩니다.

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

url = "https://namu.wiki/w/%ED%8F%AC%EC%BC%93%EB%AA%AC%EC%8A%A4%ED%84%B0/%ED%83%80%EC%9E%85"
loader = WebBaseLoader(url)
docs = loader.load()
print(f"Loaded {len(docs)} document(s) from {url}")

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)
chunks = text_splitter.split_documents(docs)
print(f"Split into {len(chunks)} chunks")

## 3. 임베딩 및 Chroma DB 구축

- 분할된 청크 데이터를 임베딩(벡터화)합니다.
- Chroma 벡터 데이터베이스에 저장하여 검색이 가능하도록 만듭니다.
- OpenAIEmbeddings를 사용하며, persist_directory로 DB를 파일로 저장할 수 있습니다.

In [4]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

database = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    collection_name='pokemon',
    persist_directory="../chroma"
)

In [ ]:
# Retriever 준비
# as_retriever()로 벡터 DB에서 유사한 문서를 검색하는 객체를 생성합니다.
retriever = database.as_retriever(search_kwargs={'k': 2})

question = "포켓몬 타입 상성에 대해 알려줘."
retrieved_docs = retriever.invoke(question)
for i, doc in enumerate(retrieved_docs, 1):
    print(f"[{i}] {doc.page_content[:200]}...\n")

## 4. PromptTemplate을 활용한 답변 생성

- PromptTemplate을 사용하면 context(검색 결과)와 question(질문)을 조합해 원하는 형태의 프롬프트를 쉽게 만들 수 있습니다.
- 아래 예시는 context와 question을 받아 이모티콘을 활용해 친근하게 답변하도록 LLM에 요청합니다.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o', temperature=0)

rag_prompt_template = PromptTemplate(
    template="""
    아래는 포켓몬 타입에 대한 정보입니다.
    이모티콘을 사용해서 친근하게 답변하세요.
    {context}

    질문: {question}
    답변:
    """,
    input_variables=["context", "question"]
)

context = "\n\n".join([doc.page_content for doc in retrieved_docs])
prompt = rag_prompt_template.format(context=context, question=question)
response = llm.invoke(prompt)
print(response.content)

## 5. LangChain Hub의 프롬프트 활용

- LangChain Hub에는 다양한 RAG(검색 증강 생성) 시나리오에 맞게 설계된 표준 프롬프트가 저장되어 있습니다.
- `"rlm/rag-prompt"`는 context(검색 결과)와 question(질문)을 받아, LLM이 참고 문맥을 활용해 답변하도록 유도하는 구조의 프롬프트입니다.
- 직접 프롬프트를 작성하지 않아도 검증된 템플릿을 바로 사용할 수 있습니다.
- hub.pull()로 프롬프트를 불러와 chain에 바로 연결할 수 있습니다.

In [ ]:
from langchain import hub

hub_prompt = hub.pull("rlm/rag-prompt")
rag_chain = hub_prompt | llm
result = rag_chain.invoke({"context": context, "question": question})
print(result.content)

## 6. RunnablePassthrough로 Chain 구성

- RunnablePassthrough는 입력값을 가공하지 않고 그대로 다음 단계로 전달할 때 사용하는 LangChain 유틸리티입니다.
- 여러 입력값 중 일부만 그대로 다음 runnable에 넘기고 싶을 때 활용할 수 있습니다.
- 아래 예시는 context는 retriever에서, question은 그대로 넘겨서 프롬프트에 전달합니다.

In [ ]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | hub_prompt
    | llm
)
result = rag_chain.invoke(question)
print(result.content)